In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/pSp_self_customize')


Mounted at /content/drive


In [2]:
import dlib
import glob

import cv2
import numpy as np


In [3]:
test_image = "image_datasets/inversion_images/brie_larson.jpg"

input_path = "image_datasets/ffhq0k_5k"
original_output_path = "image_datasets/mosaiced_ori_images"
processed_output_path = "image_datasets/mosaiced_pro_images"

try:
  os.makedirs(original_output_path)
  os.makedirs(processed_output_path)
except:
  pass


In [4]:
#Load face landmark detection
detector = dlib.get_frontal_face_detector()


In [5]:
def mosaic(img, face_rectangle, neighbor=9):
  x1 = face_rectangle.left()
  y1 = face_rectangle.top()
  w = face_rectangle.right() - face_rectangle.left()
  h = face_rectangle.bottom() - face_rectangle.top()

  for i in range(0, h, neighbor): # 从0到h，步长为neighbor
    for j in range(0, w, neighbor):
      rect = [x1+j, y1+i]
      color = img[y1+i, x1+j].tolist()
      left_up = (rect[0], rect[1])
      x2 = rect[0] + neighbor - 1
      y2 = rect[1] + neighbor - 1

      if x2 > x1+w:
        x2 = x1+w
      if y2 > y1+h:
        y2 = y1+h
      right_down = (x2,y2)
      cv2.rectangle(img, pt1=left_up, pt2=right_down, color=color, thickness=-1)

  return img


In [6]:
counts = 0
for f in glob.glob(os.path.join(input_path, "*.png")):
  #print("Processing file: {}".format(f))
  image_np = dlib.load_rgb_image(f)
  dets = detector(image_np,1)

  image_cv = cv2.imread(f)
  try:
    mosaic_img = mosaic(img=image_cv.copy(), face_rectangle=dets[0], neighbor=15)
    f_name = f.split("/")[-1]
    cv2.imwrite(os.path.join(original_output_path,f_name), image_cv)
    cv2.imwrite(os.path.join(processed_output_path,"mosaiced_"+f_name), mosaic_img)
  except:
    continue
  
  counts += 1
  if counts % 10 == 0:
        print('Already processing {} files\r'.format(counts), end='', flush=True)

print("{} images have been mosaiced!".format(counts))


4986 images have been mosaiced!


In [7]:
counts

4986